<img src="NotebookAddons/blackboard-banner.png" width="100%" />
<font face="Calibri">
<br>
<font size="5"> <b>Exploring RGB Visualzations of SAR and Deriving SAR Time Series Metrics</b></font>

<br>
<font size="4"> <b> Franz J Meyer; University of Alaska Fairbanks & Josef Kellndorfer, <a href="http://earthbigdata.com/" target="_blank">Earth Big Data, LLC</a> </b> <br>
<img style="padding:7px;" src="NotebookAddons/UAFLogo_A_647.png" width="170" align="right" /></font>

<font size="3">This notebook introduces you to some popular RGB visualizations of multi-temporal and multi-polarization SAR data. The exercise is done in the framework of *Jupyter Notebooks*. The Jupyter Notebook environment is easy to launch in any web browser for interactive data exploration with provided or new training data. Notebooks are comprised of text written in a combination of executable python code and markdown formatting including latex style mathematical equations. Another advantage of Jupyter Notebooks is that they can easily be expanded, changed, and shared with new data sets or newly available time series steps. Therefore, they provide an excellent basis for collaborative and repeatable data analysis. <br>

<b>This notebook covers the following data analysis concepts:</b>

- How to create RGB visualizations of multi-temporal SAR data
- How to interprete these RGB images
- Popular RGB visualizations of dual-polarization SAR data
- How do derive time-series metrics from deep SAR data stacks
- How to export RGB products as GeoTiffs for visualization in a GIS
</font>


</font>

<hr>
<font face="Calibri" size="5" color='rgba(200,0,0,0.2)'> <b>Important Note about JupyterHub</b> </font>
<br><br>
<font face="Calibri" size="3"> <b>Your JupyterHub server will automatically shutdown when left idle for more than 1 hour. Your notebooks will not be lost but you will have to restart their kernels and re-run them from the beginning. You will not be able to seamlessly continue running a partially run notebook.</b> </font>


In [ ]:
from IPython.display import Markdown
from IPython.display import display

env = !echo $CONDA_PREFIX
if env[0] == '':
    env[0] = 'Python 3 (base)'
if env[0] != '/home/jovyan/.local/envs/rtc_analysis':
    display(Markdown(f'<text style=color:red><strong>WARNING:</strong></text>'))
    display(Markdown(f'<text style=color:red>This notebook should be run using the "rtc_analysis" conda environment.</text>'))
    display(Markdown(f'<text style=color:red>It is currently using the "{env[0].split("/")[-1]}" environment.</text>'))
    display(Markdown(f'<text style=color:red>Select the "rtc_analysis" from the "Change Kernel" submenu of the "Kernel" menu.</text>'))
    display(Markdown(f'<text style=color:red>If the "rtc_analysis" environment is not present, use Create_OSL_Conda_Environments.ipynb to create it.</text>'))
    display(Markdown(f'<text style=color:red>Note that you must restart your server after creating a new environment before it is usable by notebooks.</text>'))

<hr>
<font face="Calibri">

<font size="5"> <b> 0. Importing Relevant Python Packages </b> </font>

<font size="3">In this notebook we will use the following scientific libraries:
<ol type="1">
    <li> <b><a href="https://pandas.pydata.org/" target="_blank">Pandas</a></b> is a Python library that provides high-level data structures and a vast variety of tools for analysis. The great feature of this package is the ability to translate rather complex operations with data into one or two commands. Pandas contains many built-in methods for filtering and combining data, as well as the time-series functionality. </li>
    <li> <b><a href="https://www.gdal.org/" target="_blank">GDAL</a></b> is a software library for reading and writing raster and vector geospatial data formats. It includes a collection of programs tailored for geospatial data processing. Most modern GIS systems (such as ArcGIS or QGIS) use GDAL in the background.</li>
    <li> <b><a href="http://www.numpy.org/" target="_blank">NumPy</a></b> is one of the principal packages for scientific applications of Python. It is intended for processing large multidimensional arrays and matrices, and an extensive collection of high-level mathematical functions and implemented methods makes it possible to perform various operations with these objects. </li>
    <li> <b><a href="https://matplotlib.org/index.html" target="_blank">Matplotlib</a></b> is a low-level library for creating two-dimensional diagrams and graphs. With its help, you can build diverse charts, from histograms and scatterplots to non-Cartesian coordinates graphs. Moreover, many popular plotting libraries are designed to work in conjunction with matplotlib. </li>
   
</font>

<font face="Calibri" size="3"> Our first step is to <b>import them:</b> </font>

In [ ]:
%%capture

import os # for chdir, getcwd, path.exists

import pandas as pd # for DatetimeIndex
from osgeo import gdal # for Info
import numpy as np # for copy, isnan, log10, ma.masked_where, max, mean, min, percentile, power, unique, var, where 
from skimage import exposure # to enhance image display

%matplotlib inline
import matplotlib.pylab as plb # for figure, grid, rcParams, savefig
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib import rc

from IPython.display import HTML


import asf_notebook as asfn
asfn.jupytertheme_matplotlib_format()

<hr>
<font face="Calibri">

<font size="5"> <b> 1. Load Data Stack</b> </font> <img src="NotebookAddons/Deforest-MadreDeDios.jpg" width="350" style="padding:7px;" align="right" /> 

<font size="3"> This notebook will be using a 78-image deep dual-polarization C-band SAR data stack over Madre de Dios in Peru to demonstrate how to create color composites from multi temporal and dual-polarization SAR data and how to derive higher level parameters such as the multi-temporal mean, the coefficient of variation, and others. The C-band data were acquired by ESA's Sentinel-1 SAR sensor constellation and are available to you through the services of the <a href="https://www.asf.alaska.edu/" target="_blank">Alaska Satellite Facility</a>. 

The site in question is interesting as it has experienced extensive logging over the last 10 years (see image to the right; <a href="https://blog.globalforestwatch.org/" target="_blank">Monitoring of the Andean Amazon Project</a>). Since the 1980s, people have been clearing forests in this area for farming, cattle ranching, logging, and (recently) gold mining. Creating RGB color composites is an easy way to visualize ongoing changes in the landscape.
</font></font>
<br><br>
<font face="Calibri" size="3">Before we get started, let's first <b>create a working directory for this analysis and change into it:</b> </font>

In [ ]:
path = "/home/jovyan/notebooks/SAR_Training/English/Hazards/data_Ex2-4_S1-MadreDeDios"
asfn.new_directory(path)
os.chdir(path)
print(f"Current working directory: {os.getcwd()}")

<font face="Calibri" size="3">We will <b>retrieve the relevant data</b> from an <a href="https://aws.amazon.com/" target="_blank">Amazon Web Service (AWS)</a> cloud storage bucket <b>using the following command</b>:</font></font>

In [ ]:
time_series_path = 's3://asf-jupyter-data/MadreDeDios.zip'
time_series = os.path.basename(time_series_path)
!aws --region=us-east-1 --no-sign-request s3 cp $time_series_path $time_series

<font face="Calibri" size="3"> Now, let's <b>unzip the file (overwriting previous extractions) and clean up after ourselves:</b> </font>

In [ ]:
if asfn.path_exists(time_series):
    asfn.asf_unzip(os.getcwd(), time_series)
    os.remove(time_series)

<br>
<font face="Calibri" size="5"> <b> 2. Define Data Directory and Path to VRT </b> </font> 
<br><br>
<font face="Calibri" size="3"><b>Create a variable containing the VRT filename and the image acquisition dates:</b></font>

In [ ]:
!gdalbuildvrt -separate raster_stack.vrt tiffs/*_VV.tiff
image_file_VV = "raster_stack.vrt"

<font face="Calibri" size="3"><b>Create an index of timedelta64 data with Pandas:</b></font>

In [ ]:
!ls tiffs/*_VV.tiff | sort | cut -c 7-21 > raster_stack_VV.dates
datefile_VV = 'raster_stack_VV.dates'
dates_VV = open(datefile_VV).readlines()
tindex_VV = pd.DatetimeIndex(dates_VV)

<font face="Calibri" size="3"><b>Print the bands and dates for all images in the virtual raster table (VRT):</b></font>

In [ ]:
print(f"Bands and dates for {image_file_VV}")
for i, d in enumerate(tindex_VV):
    print("{:4d} {}".format(i+1, d.date()), end=' ')
    if (i+1)%5 == 1:
        print()

<hr>
<br>
<font face="Calibri" size="5"> <b> 3. Open Your Data Stack and Visualize Some Layers </b> </font> 

<font face="Calibri" size="3"> We will <b>open your VRT</b> and visualize some layers using Matplotlib. </font>

In [ ]:
img = gdal.Open(image_file_VV)

<font face="Calibri" size="3"><b>Print the bands, pixels, and lines:</b></font>

In [ ]:
print(f"Number of  bands: {img.RasterCount}")
print(f"Number of pixels: {img.RasterXSize}")
print(f"Number of  lines: {img.RasterYSize}")

<font face="Calibri" size="3"><b>Read in raster data for the first two bands:</b></font>

In [ ]:
raster_1 = img.GetRasterBand(1).ReadAsArray() # change the number passed to GetRasterBand() to 
where_are_NaNs = np.isnan(raster_1)           # read rasters from different bands
raster_1[where_are_NaNs] = 0

raster_2 = img.GetRasterBand(78).ReadAsArray() #must pass a valid band number to GetRasterBand()
where_are_NaNs = np.isnan(raster_2)
raster_2[where_are_NaNs] = 0

<font face="Calibri" size="3"><b>Plot images and histograms for bands 1 and 2:</b></font>

In [ ]:
# Setup the pyplot plots
fig = plb.figure(figsize=(18, 16)) # Initialize figure with a size
ax1 = fig.add_subplot(221)  # 221 determines: 2 rows, 2 plots, first plot
ax2 = fig.add_subplot(222)  # 222 determines: 2 rows, 2 plots, second plot
ax3 = fig.add_subplot(223)  # 223 determines: 2 rows, 2 plots, third plot
ax4 = fig.add_subplot(224)  # 224 determines: 2 rows, 2 plots, fourth plot
plt.rcParams.update({'font.size': 14})

# Plot the band 1 image
band_number = 1
ax1.imshow(raster_1, cmap='gray', vmin=0, vmax=0.3) 
ax1.set_title('Image Band {} {}'.format(band_number, tindex_VV[band_number-1].date()))

# Flatten the band 1 image into a 1 dimensional vector and plot the histogram:
h = ax2.hist(raster_1.flatten(), bins=200, range=(0, 0.3))
ax2.xaxis.set_label_text('Calibrated Radar Cross Section (RCS) [Power Scale]')
ax2.set_title('Histogram Band {} {}'.format(band_number, tindex_VV[band_number-1].date()))

# Plot the band 2 image
band_number = 78
ax3.imshow(raster_2, cmap='gray', vmin=0, vmax=0.3)
ax3.set_title('Image Band {} {}'.format(band_number, tindex_VV[band_number-1].date()))

# Flatten the band 2 image into a 1 dimensional vector and plot the histogram:
h = ax4.hist(raster_2.flatten(), bins=200, range=(0, 0.3))
ax4.xaxis.set_label_text('Calibrated Radar Cross Section (RCS) [Power Scale]')
ax4.set_title('Histogram Band {} {}'.format(band_number, tindex_VV[band_number-1].date()))

<hr>
<br>
<font face="Calibri" size="5"> <b> 4. Create a Time Series Animation to get an Idea of the Dynamics at the Site </b> </font>

<font face="Calibri" size="4"> <b> 4.1 Load Time Series Stack </b> </font>

<font face="Calibri" size="3">Now we are ready to create a time series animation from the calibrated SAR data.
<br><br>
<b>First, create a raster from band 0 and a raster stack from all the images:</b>
</font> 

In [ ]:
band = img.GetRasterBand(1)
raster0 = band.ReadAsArray()
band_number = 0 # Needed for updates
rasterstack_VV = img.ReadAsArray()

<br>
<font face="Calibri" size="4"> <b> 4.2 Create Time Series Animation </b> </font>

<font face="Calibri" size="3"><b>Create and move into a directory in which to store our plots and animations:</b></font> 

In [ ]:
os.chdir(path)
product_path = 'plots_and_animations'
asfn.new_directory(product_path)
if asfn.path_exists(product_path) and os.getcwd() != f"{path}/{product_path}":
    os.chdir(product_path)
print(f"Current working directory: {os.getcwd()}")

In [ ]:
%%capture 
fig = plt.figure(figsize=(7, 7))
ax = fig.subplots()
ax.axis('off')
vmin = np.percentile(rasterstack_VV.flatten(), 5)
vmax = np.percentile(rasterstack_VV.flatten(), 95)

im = ax.imshow(raster0, cmap='gray', vmin=vmin, vmax=vmax)
ax.set_title("{}".format(tindex_VV[0].date()))

def animate(i):
    ax.set_title("{}".format(tindex_VV[i].date()))
    im.set_data(rasterstack_VV[i])

# Interval is given in milliseconds
ani = animation.FuncAnimation(fig, animate, frames=rasterstack_VV.shape[0], interval=200)

<font face="Calibri" size="3"><b>Configure matplotlib's RC settings for the animation:</b></font> 

In [ ]:
rc('animation', embed_limit=40971520.0)  # We need to increase the limit maybe to show the entire animation

<font face="Calibri" size="3"><b>Create a javascript animation of the time-series running inline in the notebook:</b></font> 

In [ ]:
HTML(ani.to_jshtml())

<font face="Calibri" size="3"><b>Delete the dummy png</b> that was saved to the current working directory while generating the javascript animation in the last code cell.</font> 

In [ ]:
try:
    os.remove('None0000000.png')
except FileNotFoundError:
    pass

<font face="Calibri" size="3"><b>Save the animation (animation.gif):</b> </font> 

In [ ]:
ani.save('animation.gif', writer='pillow', fps=2)

<hr>
<br>
<font face="Calibri" size="5"> <b> 5. Create RGB Visualization from Multi-Temporal SAR Images</b> </font>

<font face="Calibri" size="3">Now we are ready to create our first RGB visualization. We will pick images from different dates/times/years to form an RGB. The colors in the RGB will then provide information on changes that occurred during the time span covered by the data.
<br><br>
<b>First, we mask out pixels without relevant information to be unbiased in statical number we calculate later:</b>
</font> 

In [ ]:
mask = rasterstack_VV == 0
rasterPwr = np.ma.array(rasterstack_VV, mask=mask, dtype=np.float32)

<br>
<font face="Calibri" size="3">We make an RGB stack to display the first, center, and last time step of our available stack as a multi-temporal color composite. The np.dstack results in an array of the form [lines,pixels,bands], which is the format we need for RGB display with matplotlib's imshow() function. Note that numpy array indexing starts with 0, so band 1 is raster[0].</font> 

In [ ]:
rgb_bands = (1, int(img.RasterCount/2), img.RasterCount)  # first, center, last band

rgb_idx = np.array(rgb_bands)-1  # get array index from bands by subtracting 1

rgb = np.dstack((rasterPwr[rgb_idx[0]], 
                 rasterPwr[rgb_idx[1]], 
                 rasterPwr[rgb_idx[2]]))

rgb_dates = (tindex_VV[rgb_idx[0]].date(),
             tindex_VV[rgb_idx[1]].date(), 
             tindex_VV[rgb_idx[2]].date())

<br>
<font face="Calibri" size="3">We are also interested in displaying the image enhanced with histogram equalization. We can use the function exposure.equalize_hist() from the skimage.exposure module.</font> 

In [ ]:
rgb_stretched = rgb.copy()
# For each band we apply the strech
for i in range(rgb_stretched.shape[2]):
    rgb_stretched[:,:,i] = exposure.\
    equalize_hist(rgb_stretched[:,:,i].data,
    mask =~ np.equal(rgb_stretched[:,:,i].data,0.))

<br>
<font face="Calibri" size="3">Now let's <b>display the the unstrechted and histogram equalized images side by side.</b></font> 

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16, 8))
fig.suptitle('Multi-temporal Sentinel-1 backscatter image R:{} G:{} B:{}'
             .format(rgb_dates[0],rgb_dates[1],rgb_dates[2]))
plt.axis('off')
ax[0].imshow(rgb)
ax[0].set_title('Unstreched')
ax[0].axis('off')
ax[1].imshow(rgb_stretched)
ax[1].set_title('Histogram Equalized')
_ = ax[1].axis('off')

<br>
<div class="alert alert-success">
<font face="Calibri" size="5"> <b> <font color='rgba(200,0,0,0.2)'> <u>EXERCISE</u>:  </font> Pick Different Time Steps and Interpret Resulting RGB Visualization </b> </font>

<font face="Calibri" size="3"> To get a feeling for the RGB visualizations, please explore the 78 image deep data stack a bit more. Pick different mutli-temporal image layers and form more RGB visualizations. Interpret the colors you see in terms of the change that likely occurred on the ground.
</font>
</div>
<br>
<hr>

<font face="Calibri" size="3">Ideally, you may want to analyze these images in a GIS framework, where you can combine the SAR RGBs with other relevant data layers. To do so, we will export <font color='rgba(200,0,0,0.2)'><b>your favorite multi-temporal RGB image</b></font> as a GeoTiff. First, we <b>write a function to convert our plots into GeoTiffs:</b></font> 

In [ ]:
vrt = gdal.Open(f"{path}/{image_file_VV}")
vrt_info = gdal.Info(vrt, format='json')
coords = [vrt_info['cornerCoordinates']['upperLeft'], 
          vrt_info['cornerCoordinates']['lowerRight']]
print(coords)

In [ ]:
utm_zone = vrt_info['coordinateSystem']['wkt'].split('EPSG",')[-1][0:-2]

In [ ]:
# do not include a file extension in out_filename
# extent must be in the form of a list: [[upper_left_x, upper_left_y], [lower_right_x, lower_right_y]]
def geotiff_from_plot(source_image, out_filename, extent, utm_zone, cmap=None, vmin=None, vmax=None, interpolation=None, dpi=300):
    plt.figure()
    plt.axis('off')
    plt.imshow(source_image, cmap=cmap, vmin=vmin, vmax=vmax, interpolation=interpolation)
    temp = f"{out_filename}_temp.png"
    plt.savefig(temp, dpi=dpi, transparent='true', bbox_inches='tight', pad_inches=0)

    cmd = f"gdal_translate -of Gtiff -a_ullr {extent[0][0]} {extent[0][1]} {extent[1][0]} {extent[1][1]} -a_srs EPSG:{utm_zone} {temp} {out_filename}.tiff"
    !{cmd}
    try:
        os.remove(temp)
    except FileNotFoundError:
        pass

<font face="Calibri" size="3"><b>Now we can save your last RGB image as a GeoTIFF (MadreDeDios-RGB.tiff):</b></font> 

In [ ]:
%%capture
geotiff_from_plot(rgb_stretched, 'MadreDeDios-multitemp-RGB', coords, utm_zone)

<hr>
<br>
<font face="Calibri" size="5"> <b> 6. Create RGB Visualization from Dual-Pol SAR Images</b> </font>

<font face="Calibri" size="3">Now let's demonstrate another popular RGB visualization, one that is particularly useful for dual-pol SAR data such as Sentinel-1. The color composites are constructed with co-polarized (VV-channel) data assigned to the red channel, cross-polarized (VH) data to the green channel, and the co-/cross-polarized ratio (VV/VH ratio) to the blue channel. A nice effect for forest applications with this color assignment strategy is that forests tend to be shown in shades of green, and typically the brightness of green corresponds to the amount of biomass in the forest. Also, water tends to be represented in blue colors, which also represents other surface scattering components. Naturally, different histogram stretches may be applied to enhance various surface components.
<br><br>
<b>To create this visualization, we first need to load the cross-polarized VH data into the notebook:</b>
</font> 

In [ ]:
os.chdir(path)
!gdalbuildvrt -separate raster_stack_VH.vrt tiffs/*_VH.tiff
image_file_VH = "raster_stack_VH.vrt"

In [ ]:
!ls tiffs/*_VH.tiff | sort | cut -c 7-21 > raster_stack_VH.dates
datefile_VH = 'raster_stack_VH.dates'
dates_VH = open(datefile_VH).readlines()
tindex_VH = pd.DatetimeIndex(dates_VH)

In [ ]:
if asfn.path_exists(image_file_VH):
    print(f"Bands and dates for {image_file_VH}")
    for i, d in enumerate(tindex_VH):
        print("{:4d} {}".format(i+1, d.date()),end=' ')
        if (i+1)%5 == 1: print()

<font face="Calibri" size="3"><b>Now we load the VH data stack and prepare it for visualization:</b></font> 

In [ ]:
img_VH = gdal.Open(image_file_VH)
band_VH = img_VH.GetRasterBand(1)
raster0_VH = band_VH.ReadAsArray()
band_number = 0 # Needed for updates
rasterstack_VH = img_VH.ReadAsArray()
mask = rasterstack_VH == 0
rasterPwr_VH = np.ma.array(rasterstack_VH, mask=mask, dtype=np.float32)

<font face="Calibri" size="3"><b><font color='rgba(200,0,0,0.2)'>Pick a band number</font> you would like to visualize:</b></font> 

In [ ]:
#Pick a band number for visualization
bandno = 1

<font face="Calibri" size="3"><b>Now we can create an RGB image using the mentioned color assignments (R: VV; G: VH; B: VV/VH), color stretch the RGB, and visualize it:</b></font> 

In [ ]:
rgb_band = (bandno, bandno, bandno)  # first, center, last band
rgb_idx = np.array(rgb_bands)-1  # get array index from bands by subtracting 1
rgb = np.dstack((rasterPwr[rgb_idx[0]], rasterPwr_VH[rgb_idx[0]], rasterPwr[rgb_idx[0]]/rasterPwr_VH[rgb_idx[0]]))
rgb_date = (tindex_VH[rgb_idx[0]].date())

In [ ]:
rgb_stretched_POL = rgb.copy()
# For each band we apply the strech
for i in range(rgb_stretched_POL.shape[2]):
    rgb_stretched_POL[:,:,i] = exposure.\
    equalize_hist(rgb_stretched_POL[:,:,i].data,
    mask =~ np.equal(rgb_stretched_POL[:,:,i].data,0.))

In [ ]:
fig = plb.figure(figsize=(16, 16)) # Initialize figure with a size
ax1 = fig.add_subplot(221)  # 221 determines: 2 rows, 2 plots, first plot
ax2 = fig.add_subplot(222)  # 222 determines: 2 rows, 2 plots, second plot
ax3 = fig.add_subplot(223)  # 223 determines: 2 rows, 2 plots, third plot
ax4 = fig.add_subplot(224)  # 224 determines: 2 rows, 2 plots, fourth plot
plt.rcParams.update({'font.size': 14})

# Plot the VV band
r_1 = img.GetRasterBand(bandno).ReadAsArray()
vmin = np.percentile(r_1.flatten(), 5)
vmax = np.percentile(r_1.flatten(), 95)
ax1.imshow(r_1, cmap='gray', vmin=vmin, vmax=vmax) 
ax1.set_title('VV Channel')
ax1.axis('off')

# Plot the VH band
r_1 = img_VH.GetRasterBand(bandno).ReadAsArray()
vmin = np.percentile(r_1.flatten(), 5)
vmax = np.percentile(r_1.flatten(), 95)
ax2.imshow(r_1, cmap='gray', vmin=vmin, vmax=vmax)
ax2.set_title('VH Channel')
ax2.axis('off')

# Plot the VV/VH band
r_1 = img.GetRasterBand(bandno).ReadAsArray()/img_VH.GetRasterBand(bandno).ReadAsArray()
vmin = np.percentile(r_1.flatten(), 5)
vmax = np.percentile(r_1.flatten(), 95)
ax3.imshow(r_1, cmap='gray', vmin=vmin, vmax=vmax)
ax3.set_title('VV/VH Ratio Channel')
ax3.axis('off')

# Plot the RGB Composite band
ax4.imshow(rgb_stretched_POL)
ax4.set_title('Color Composite')
ax4.axis('off')

<font face="Calibri" size="3">We will again <b>export the RGB composite as a GeoTiff to allow further analysis in your favorite GIS environment:</b></font> 

In [ ]:
%%capture
geotiff_from_plot(rgb_stretched_POL, 'plots_and_animations/MadreDeDios-multipol-RGB', coords, utm_zone)

<br>
<div class="alert alert-success">
<font face="Calibri" size="5"> <b> <font color='rgba(200,0,0,0.2)'> <u>EXERCISE</u>:  </font> Pick Different Time Steps and Interpret Resulting Dual-Pol RGB Images </b> </font>

<font face="Calibri" size="3"> Explore the 78 image deep data stack a bit more. Pick different time steps in the data stack and create additional RGB composites. You may see interesting changes in the color patterns. Answer the following questions for yourself:

<ul>
  <li>What are the green areas and what are different shades of green mean?</li>
  <li>Why is there fewer green in the visualization than you might have expected?</li>
  <li>What are the blue-shaded regions?</li>
  <li>What kind of differences in color patterns do you see between different time steps? What might be the reasons for these changes?</li>
</ul>

</font>
</div>
<br>
<hr>

<br>
<hr>
<font face="Calibri" size="5"> <b> 7. Plot the Time Series of Means Calculated Across the Subset </b> </font>

<font face="Calibri" size="3"> To create the time series of means, we will go through the following steps:
1. Ensure that you use the data in **power scale** ($\gamma^o_{pwr}$) for your mean calculations.
2. compute means.
3. convert the resulting mean values into dB scale for visualization.
4. plot time series of means. </font> 
<br><br>
<font face="Calibri" size="3"> <b>Compute the means:</b> </font>

In [ ]:
rs_means_pwr_VH = np.mean(rasterPwr_VH, axis=(1, 2))
rs_means_pwr_VV = np.mean(rasterPwr, axis=(1, 2))

<font face="Calibri" size="3"><b>Convert resulting mean value time-series to dB scale for visualization:</b></font>

In [ ]:
rs_means_dB_VH = 10.*np.log10(rs_means_pwr_VH)
rs_means_dB_VV = 10.*np.log10(rs_means_pwr_VV)

<font face="Calibri" size="3"><b>Plot and save the time series of means (RCSoverTime.png):</b></font>

In [ ]:
# 3. Now let's plot the time series of means
plt.rcParams.update({'font.size': 14})
fig = plt.figure(figsize=(16, 4))
ax1 = fig.add_subplot(111)
ax1.plot(tindex_VV, rs_means_dB_VV)
ax1.set_xlabel('Date')
ax1.set_ylabel('VV Channel $\overline{\gamma^o}$ [dB]')


ax2 = ax1.twinx()
ax2.plot(tindex_VH, rs_means_dB_VH, color='red')
ax2.set_ylabel('VH Channel $\overline{\gamma^o}$ [dB]')
#fig.legend(['VV Channel', 'VH Channel'], loc=1)
fig.legend(['VV Channel', 'VH Channel'], loc=3, bbox_to_anchor=(0.07, 0.18))
plt.title('Time series profile of average band backscatter $\gamma^o$ ')
plt.savefig('time_series_means', dpi=72, transparent='true')

<hr>
<br>
<font face="Calibri" size="5"> <b> 8. Computation and Visualization of Time Series Metrics</b> </font>

<font face="Calibri" size="3">Once a time-series was constructed, we can compute <b>a set of metrics</b> that will aid us later in applications such as <i>change detection and active agriculture detection</i>. In the next code cells, we will compute the following variables for each pixel in the stack:

- Mean 
- Median
- Maximum
- Minimum
- Range (Maximum - Minimum)
- 5th Percentile
- 95th Percentile
- PRange (95th - 5th Percentile)
- Variance
- Coefficient of Variation (Variance/Mean)

<hr>
Let's first <b>define a function for calculating the intended time-series metrics</b>:
</font> 

In [ ]:
def timeseries_metrics(raster, ndv=0): 
    # Make use of numpy nan functions
    # Check if type is a float array
    if not raster.dtype.name.find('float') > -1:
        raster = raster.astype(np.float32)
    # Set ndv to nan
    if ndv != np.nan:
        raster[np.equal(raster,ndv)] = np.nan
    # Build dictionary of the metrics
    tsmetrics = {}
    rperc = np.nanpercentile(raster, [5, 50, 95], axis=0)
    tsmetrics['mean'] = np.nanmean(raster, axis=0)
    tsmetrics['max'] = np.nanmax(raster, axis=0)
    tsmetrics['min'] = np.nanmin(raster, axis=0)
    tsmetrics['range'] = tsmetrics['max'] - tsmetrics['min']
    tsmetrics['median'] = rperc[1]
    tsmetrics['p5'] = rperc[0]
    tsmetrics['p95'] = rperc[2]
    tsmetrics['prange'] = rperc[2] - rperc[0]
    tsmetrics['var'] = np.nanvar(raster, axis=0)
    tsmetrics['CV'] = tsmetrics['var'] / tsmetrics['mean']
    return tsmetrics

In [ ]:
metrics = timeseries_metrics(rasterPwr.filled(np.nan), ndv=np.nan)

In [ ]:
metric_keys = list(metrics.keys())
print(metric_keys)

<font face="Calibri" size="3">Let's look at the histograms for the time series variance and coeficient of variation to aid displaying those images:</font> 

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16, 4))
ax[0].hist(metrics['var'].flatten(), bins=100, range=(0, 0.001))
ax[1].hist(metrics['CV'].flatten(), bins=100, range=(0, 0.01))
_ = ax[0].set_title('Variance')
_ = ax[1].set_title('Coefficient of Variation')

<font face="Calibri" size="3">We use thresholds determined from those histograms to set the scaling in the time series visualization. For the backscatter metrics we choose a typical range appropriate for this ecosystem and radar sensor. A typical range is -30 dB (0.0001) to -5.2 dB (0.3).</font> 

In [ ]:
# List the metrics keys you want to plot
fig = plt.figure(figsize=(16, 40))
for i, key in enumerate(metric_keys):
    ax = fig.add_subplot(5, 2, i+1)
    if key == 'var': 
        vmin, vmax = (0.0, 0.001)
    elif key == 'CV':
        vmin, vmax = (0.0, 0.004)
    else:
        vmin, vmax = (0.0001, 0.3)
    ax.imshow(metrics[key], vmin=vmin, vmax=vmax, cmap='gray')
    ax.set_title(key.upper())
    ax.axis('off')

<font face="Calibri" size="3">We will again <b>export the RGB composite as a GeoTiff to allow further analysis in your favorite GIS environment:</b></font> 

In [ ]:
%%capture
Names = [] # List to keep track of all the names
for i in metric_keys:
    # Name, Array, DataType, NDV,bandnames=None,ref_image
    Name = os.path.join(path, f'plots_and_animations/MadreDeDios-'+i)
    print(Name)
    if i == 'var': 
        vmin, vmax = (0.0, 0.001)
    elif i == 'CV': 
        vmin, vmax = (0.0, 0.004)
    else:
        vmin, vmax = (0.0001, 0.3)
    geotiff_from_plot(metrics[i], Name, coords, utm_zone, cmap='gray', vmin=vmin, vmax=vmax)

<hr>
<br>
<font face="Calibri" size="5"> <b> 8. Conclusion</b> </font>

<font face="Calibri" size="3">RGB Visualizations can be a powerful tool to visually analyze multi-temporal or multi-polarization SAR images. For multi-temporal data, the different color bands of an RGB composite relate to different time steps; hence, color tones represent changes of the landscape over time. 
    
For multi-polarization data, the RGB representation gives some indication of the type of surface cover present in an area. Areas shown in green in these composites usually are areas of high biomass while areas in blue are often regions covered in water. 

Please use caution when interpreting multi-polarization composites made from Sentinel-1 dual-pol data. As Sentinel-1 does not penetrate deep into vegetation, the composites often are not a correct representation of biomass in an area (composites don't look as green as they should).  

For a bit more information on change detection and SAR in general, please look at the recently published <a href="https://gis1.servirglobal.net/TrainingMaterials/SAR/SARHB_FullRes.pdf" target="_blank">SAR Handbook: Comprehensive Methodologies for Forest Monitoring and Biomass Estimation</a>.
</font> 
<hr>
<br>

<font face="Calibri" size="2"> <i>Exercise2-RGBandMetricsVisualization.ipynb - Version 1.4.0 - April 2021 </i>
    <br>
    <b>Version Changes</b>
    <ul>
        <li>namespace asf_notebook</li>
    </ul>
</font>